In [1]:
import json

In [2]:
file_name = ''
with open(file_name, 'r') as df:
    data = json.load(df)

In [4]:
decode_or = []

for doc in data:
    decode_or.append(doc['decode_or'])
    doc.pop('decode_or')

for i in range(len(data)):
    data[i]['decode_or'] = decode_or[i]

In [9]:
import os
import openai
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
openai.api_key = ''

In [10]:
## Def prompt template: 1. return scores and reasons; 2. only return scores
def prompt_construct(data, with_reason=True):
    keys = [key for key in data.keys() if key not in ['summary', 'id']]
    begining = 'Given a document and some summaries, please evaluate these summaries with scores ranging from 0 to 100.'
    context = begining + f"'{keys[0]}':" + f"'{data[keys[0]]}'" + '\n'
    keys = keys[1:]
    context += f"'{keys[-1]}':" + f"'{data[keys[-1]]}'" + '\n'
    keys = keys[:-1]
    for key in keys:
        context += f"'{key}':" + f"'{data[key]}'" + '\n'
    end = 'Within summaries, please ignore the incomplete parts and the duplicate part, duplication and incompleteness will not diminish the score. Only return scores and the reason to score, sentence by sentence, with this pattern:  summary_index:  $$Score$$: , $$Reason$$: .'
    end_without_reason = 'Within summaries, please ignore the incomplete parts and the duplicate part, duplication and incompleteness will not diminish the score. Only return scores and don not return the reason sentence by sentence with this pattern:  $$summary_index$$:  $$Score$$:'
    if with_reason == True:
        prompt_template = context + end
    else:
        prompt_template = context + end_without_reason
    return prompt_template

In [ ]:
prompt_template = prompt_construct(data[0], with_reason=False)
completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": prompt_template}
  ]
)
print(completion["choices"][0]["message"]["content"])